In [14]:
print("Ok")

Ok


In [15]:
%pwd

'c:\\Code\\End-to-End-Medical-Chatbot'

In [16]:
import os
os.chdir("../")

In [17]:
%pwd

'c:\\Code'

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [13]:
extracted_data=load_pdf_file(data=r"C:\Code\new_projects\AyurSutra_Panchakarma_first\Data")

In [10]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader(
    file_path="../Data/panchkarmaBook.pdf",
    mode="elements"   # use elements for structured parsing
)
docs = loader.load()

print(docs[0].page_content[:500])

c:\Users\busin\anaconda3\envs\ayurvedbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [8]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("../Data/panchkarmaBook.pdf", strategy="ocr_only")
docs = loader.load()

print(docs[0].page_content[:500])


ModuleNotFoundError: No module named 'pdfminer'

from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("../Data/panchkarmaBook.pdf", strategy="ocr_only")
docs = loader.load()

print(docs[0].page_content[:500])


In [14]:
extracted_data

[Document(metadata={'producer': 'pdfmake', 'creator': 'pdfmake', 'creationdate': '2025-09-06T08:11:57+00:00', 'source': 'C:\\Code\\new_projects\\AyurSutra_Panchakarma_first\\Data\\pc1.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='& PRACTICAL HANDBOOK OF PANCHAKARMA PROCEDURES CHIEF EDITOR Prof. G.S. Lavekar \nDirector General Central Council for Research in Ayurveda and Siddha, New Delhi \nCONTRIBUTORS Dr. T.V. Menon Ex. Assistant Director (Ayurveda) Incharge Central Research \nInstitute (Ayurveda), Cheruthuruthy Dr. (Mrs.) Bharti Assistant Director (Ayurveda) Central \nResearch Institute (Ayurveda), New Delhi CENTRAL COUNCIL FOR RESEARCH IN AYURVEDA \nAND SIDDHA Department of AYUSH, Ministry of Health & Family Welfare Government of India \nNew Delhi - 110058 www.ccras.nic.in 2009 \x00\n\x00\nA PRACTICAL HANDBOOK OF PANCHAKARMA PROCEDURES EDITORS Dr. M.M. Padhi Deputy \nDirector (Technical), CCRAS, New Delhi Dr. Sharda Ota Research Officer (Ayurveda), CCRAS, New

In [5]:
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  0


In [41]:
from langchain.embeddings import HuggingFaceEmbeddings

In [42]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [48]:
#embeddings = download_hugging_face_embeddings()

In [ ]:
#pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [49]:
embeddings = download_hugging_face_embeddings()

In [50]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [52]:
#query_result

In [62]:
import os
os.chdir("./End-to-End-Medical-Chatbot/")
print(os.getcwd())

c:\Code\End-to-End-Medical-Chatbot


In [90]:
from dotenv import load_dotenv
load_dotenv()

True

In [91]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [67]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,            
    metric="cosine",            
    spec=ServerlessSpec(
            cloud= "aws",     
            region= "us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-1kgnu9z.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [92]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [69]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(documents=text_chunks, index_name=index_name, embedding=embeddings)

In [70]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)

In [71]:
docsearch

In [72]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [73]:
retrieved_docs = retriever.invoke("What is Acne?")

In [74]:
retrieved_docs

[Document(id='3ef0804f-5c38-4667-b55e-d382d7c8fc66', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Code\\End-to-End-Medical-Chatbot\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='7a875e1b-ebd0-4b7b-8c36-8d75d01c6f78', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Code\\End-to-End-Medical-Chatbot\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biop

In [140]:
#from langchain_openai import OpenAI
#llm = OpenAI(temperature=0.4, max_tokens=500)
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key= GEMINI_API_KEY,  # or gemini-1.5-flash for faster/lighter
    temperature=0.4,
    max_output_tokens=500
)

In [142]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. keep the "
    "answer concise."
    "\n\n"
    "{context}\n\n"
    "Final Answer:"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [147]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [148]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back.  It occurs when skin pores become clogged with oil, dead skin cells, and bacteria.  Acne vulgaris is the medical term for common acne.


In [145]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

Based on the provided text, there is no information about "stats".  I don't know.


In [146]:
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders caused by an abnormal release of a chemical from the pituitary gland, leading to increased bone and soft tissue growth.  Gigantism occurs before bone growth stops, resulting in unusual height. Acromegaly occurs after bone growth stops, causing other symptoms.
